In [782]:
# default_exp encryption

# Encryption

> Functions for encrypting files.

In [783]:
# export
import cryptography
from cryptography.fernet import Fernet
from fastcore.all import *
from pathlib import Path
from functools import partial
from shutil import copyfile
import os

In [784]:
#hide
from nbdev.showdoc import *

## Utility Functions

In [786]:
# export
def process_dirs(p:Path,fn=None,invalid_exts='mca'):
    p=Path(p)
    if p.is_file(): return fn(p,invalid_exts=invalid_exts)
    for dirpath,dirnames,filenames in os.walk(str(p)):
        if len(dirnames)==0 and len(filenames)==0: continue
        for f in filenames: fn(Path(dirpath)/Path(f),invalid_exts=invalid_exts)

In [787]:
process_dirs('test_save',lambda x,invalid_exts:print(x))

test_save/forcedchunks.dat
test_save/multipart.dat
test_save/session.lock
test_save/level.dat_old
test_save/level.dat
test_save/DIM-17/forcedchunks.dat
test_save/DIM-17/multipart.dat
test_save/DIM-17/data/villages_wyvern_lair.dat
test_save/DIM-17/data/capabilities.dat
test_save/DIM-17/data/buildcraft_wire_systems.dat
test_save/minecolonies/colonies.dat
test_save/minecolonies/colonies-2020-12-07_23.04.22.zip
test_save/data/Village.dat
test_save/data/mocreatures.dat
test_save/data/buildcraft_marker_path.dat
test_save/data/Mineshaft.dat
test_save/data/buildcraft_marker_volume.dat
test_save/data/villages.dat
test_save/data/capabilities.dat
test_save/data/buildcraft_wire_systems.dat
test_save/structurize/structurize_linksession.dat
test_save/DIM1/forcedchunks.dat
test_save/DIM1/multipart.dat
test_save/DIM1/data/villages_end.dat
test_save/DIM1/data/capabilities.dat
test_save/DIM1/data/buildcraft_wire_systems.dat
test_save/region/r.0.0.mca
test_save/region/r.0.-1.mca
test_save/DIM-1/forcedchu

## Crypto Functions

In [788]:
%%writefile test_file.txt
Hello
This is
A file that 
needs to be encrypted

Overwriting test_file.txt


In [789]:
# export
pv=lambda msg,v:(print(msg) if v else None)

def encrypt_file(p:Path,to_p:Path=None,block_sz=65536,verbose=False,prefix='encrypted',key=None,
                invalid_exts=''):
    p=Path(p)
    to_p=ifnone(to_p,Path(prefix)/p)        
    key=ifnone(key,Fernet.generate_key())
    encryptor=Fernet(key)
    os.makedirs(str(Path(to_p).parent),exist_ok=True)
    
    if p.suffix[1:] in invalid_exts.split(','): 
        copyfile(str(p),str(to_p))
        return key 
    
    with open(p,'rb') as f,open(to_p,'wb') as to_f:
        fb=f.read(block_sz)
        while len(fb)>0: 
            token=encryptor.encrypt(fb)
            pv(token,verbose)
            to_f.write(token)
            fb=f.read(block_sz) 
    return key

In [790]:
k=encrypt_file('test_file.txt',block_sz=8,verbose=True);k

b'gAAAAABfzxiKdaQeOgRTueb5wNxqrEqp-3f484q0Rr1AK4bO3xxrmNhl_DblVTMIoSN-mlw2d_yKZwH_6E1b0welPMSxzKQGKQ=='
b'gAAAAABfzxiKw0sVSvPE8d8tQ37f4oAt9uorLYOdq4ejTSa0kepmoZlLs9iL9i-dDN4RESO3uksBII0nohfGTQFinY-rOxjjJw=='
b'gAAAAABfzxiKXiIiqgz5nnUJat3o1PIp9i94dH4s5Qpt_2gJxhAnRJsnJZze1aY1MTYVc3vKgW22R2it6_e4yQo94iVWIw4yRQ=='
b'gAAAAABfzxiKPzoiCJiiVi_PO2o7dY_AmXreop-jLQFEaJu4NW_aTmjxtSzkbh9Wx-DcwXDY5lhESHEa0ui2F3sl5iUuFm_9xg=='
b'gAAAAABfzxiKzhcHjZY1VbH7Uv_NDmpj2f1JnGvh1ZG-0uJgEkf9-thGRz2aLKbOt_oaxlLLLt_UUfckrbVxXFPnUTzTMl8tSw=='
b'gAAAAABfzxiK1isupbE8vhB_OEIrviKwqTTCbbFJE92eO6c4r_2hWE6ZWExB6yyNlbVnCJtVQbolrWaywPs9UPy68OmBU9Y6cA=='
b'gAAAAABfzxiKuzahvnEX1Sr-IOPK2iX9BH_KtzEti89E8kKiLxGUQrI9Kh-Wu4sL_5yZDdp6BHeOU-pei1DqMsWW9u2losIrNQ=='


b'J2RF3E6MfqO-WVs_0CtQ7DS6LIeA2tMtny6GqK3xDLE='

In [791]:
# export
def decrypt_file(p:Path,key,to_p:Path=None,block_sz=65536,verbose=False,prefix='decrypted',
                invalid_exts=''):
    p=Path(p)
    pv(p,verbose)
    to_p=ifnone(to_p,Path(prefix)/p)
    decryptor=Fernet(key)
    os.makedirs(str(Path(to_p).parent),exist_ok=True)
    if p.suffix[1:] in invalid_exts.split(','): 
        copyfile(str(p),str(to_p))
        return key 
    
    with open(p,'rb') as f,open(to_p,'wb') as to_f:
        fb=f.read(block_sz)
        while len(fb)>0: 
            token=decryptor.decrypt(fb)
            to_f.write(token)
            fb=f.read(block_sz) 
    return key

In [792]:
decrypt_file('encrypted/test_file.txt',k,'test_file_decrypted.txt',verbose=True)

encrypted/test_file.txt


b'J2RF3E6MfqO-WVs_0CtQ7DS6LIeA2tMtny6GqK3xDLE='

In [793]:
k=Fernet.generate_key()
process_dirs('test_save',partial(encrypt_file,key=k,prefix='minecraft_encrypted'))

In [794]:
process_dirs('minecraft_encrypted/test_save',partial(decrypt_file,key=k,verbose=True))

minecraft_encrypted/test_save/forcedchunks.dat
minecraft_encrypted/test_save/multipart.dat
minecraft_encrypted/test_save/session.lock
minecraft_encrypted/test_save/level.dat_old
minecraft_encrypted/test_save/level.dat
minecraft_encrypted/test_save/DIM-17/forcedchunks.dat
minecraft_encrypted/test_save/DIM-17/multipart.dat
minecraft_encrypted/test_save/DIM-17/data/villages_wyvern_lair.dat
minecraft_encrypted/test_save/DIM-17/data/capabilities.dat
minecraft_encrypted/test_save/DIM-17/data/buildcraft_wire_systems.dat
minecraft_encrypted/test_save/minecolonies/colonies.dat
minecraft_encrypted/test_save/minecolonies/colonies-2020-12-07_23.04.22.zip
minecraft_encrypted/test_save/data/Village.dat
minecraft_encrypted/test_save/data/mocreatures.dat
minecraft_encrypted/test_save/data/buildcraft_marker_path.dat
minecraft_encrypted/test_save/data/Mineshaft.dat
minecraft_encrypted/test_save/data/buildcraft_marker_volume.dat
minecraft_encrypted/test_save/data/villages.dat
minecraft_encrypted/test_sav

In [795]:
# hide
from nbdev.export import *
from nbdev.export2html import *
from nbdev.cli import make_readme
make_readme()
notebook2script()
notebook2html()

converting /opt/project/play_it_later/nbs/index.ipynb to README.md
Converted 00_encryption.ipynb.
Converted index.ipynb.


converting: /opt/project/play_it_later/nbs/00_encryption.ipynb
